# BDA2 - Spark SQL

**Submission by:**
- Julius Kittler (julki092)
- Maximilian Pfundstein (maxpf364)

Redo the exercises BDA1 using Spark SQL whenever possible. There are two ways to write queries in SparkSQL - using built-in API functions or running SQL-like
queries. To pass this lab, you need to use built-in API functions for all the questions, in addition, SQL-like queries for the second question. The slides of this link (https://www.ida.liu.se/~732A54/lab/SparkSQLQuickIntro.pdf) show some examples of Spark SQL.

## Setup

The following code is used to prepare the data files, execute only once and **be careful!**

In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import SQLContext, Row
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import col
import csv
import math
import sys
import pandas as pd

tr_path = "../../../data/temperature-readings.csv"
tr_big_path = "../../../data/temperature-readings-big.csv"
pr_path = "../../../data/precipitation-readings.csv"
sro_path = "../../../data/stations-Ostergotland.csv"
sr_path = "../../../data/stations.csv"

In [2]:
sc = pyspark.SparkContext(appName = "Temperature")
sc_sql = pyspark.SQLContext(sc)

temperature_readings = sc.textFile(tr_path)
temperature_readings_big = sc.textFile(tr_big_path)
precipitation_readings = sc.textFile(pr_path)
station_oster_readings = sc.textFile(sro_path)
station_readings = sc.textFile(sr_path)

## Assignment 1

What are the lowest and highest temperatures measured each year for the period 1950- 2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the `temperature-readings.csv` file.

### From Previous Lab: Min and Max for each station

In [3]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014)).groupBy('year')
res = res.agg(F.max('temp').alias('temp_max'), F.min('temp').alias('temp_min'))

global df

df = res.orderBy('temp_max', ascending=False).toPandas()

CPU times: user 61.4 ms, sys: 29 ms, total: 90.4 ms
Wall time: 3min 15s


In [4]:
df.head(10)

,year,temp_max,temp_min
0,1975,36.1,-37.0
1,1992,35.4,-36.1
2,1994,34.7,-40.5
3,2014,34.4,-42.5
4,2010,34.4,-41.7
5,1989,33.9,-38.2
6,1982,33.8,-42.2
7,1968,33.7,-42.0
8,1966,33.5,-49.4
9,1983,33.3,-38.2


**Task 1a:** What are the lowest and highest temperature measured each year for the period 1950 to 2014? Provide the list sorted in the descending order with respect to the maximum temperature. Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

### Min and Max for each station including the different station numbers

In [5]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))

w = Window.partitionBy('year')

res_max = res.withColumn('temp_max', F.max('temp').over(w)).where(F.col('temp') == F.col('temp_max')).drop('temp')
res_min = res.withColumn('temp_min', F.min('temp').over(w)).where(F.col('temp') == F.col('temp_min')).drop('temp')

global df

df = res_max.join(res_min, "year").orderBy(['temp_max', 'year'], ascending=False).toPandas()

CPU times: user 70.7 ms, sys: 35.6 ms, total: 106 ms
Wall time: 4min 32s


In [6]:
df.head(10)

,year,station,temp_max,station,temp_min
0,1975,86200,36.1,157860,-37.0
1,1992,63600,35.4,179960,-36.1
2,1994,117160,34.7,179960,-40.5
3,2014,96560,34.4,192840,-42.5
4,2010,75250,34.4,191910,-41.7
5,1989,63050,33.9,166870,-38.2
6,1982,94050,33.8,113410,-42.2
7,1968,137100,33.7,179950,-42.0
8,1966,151640,33.5,179950,-49.4
9,2002,78290,33.3,169860,-42.2


### year, station with the max, maxValue ORDER BY maxValue DESC

In [7]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))

w = Window.partitionBy('year')

res_max = res.withColumn('temp_max', F.max('temp').over(w)).where(F.col('temp') == F.col('temp_max')).drop('temp')
global df

df = res_max.orderBy(['temp_max', 'year'], ascending=False).toPandas()

CPU times: user 77.8 ms, sys: 10.7 ms, total: 88.5 ms
Wall time: 3min 51s


In [8]:
df.head(10)

,station,year,temp_max
0,86200,1975,36.1
1,63600,1992,35.4
2,117160,1994,34.7
3,96560,2014,34.4
4,75250,2010,34.4
5,63050,1989,33.9
6,94050,1982,33.8
7,137100,1968,33.7
8,151640,1966,33.5
9,78290,2002,33.3


### year, station with the min, minValue ORDER BY minValue DESC

In [9]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))

w = Window.partitionBy('year')

res_min = res.withColumn('temp_min', F.min('temp').over(w)).where(F.col('temp') == F.col('temp_min')).drop('temp')
global df

df = res_min.orderBy(['temp_min', 'year'], ascending=False).toPandas()

CPU times: user 75.4 ms, sys: 14.3 ms, total: 89.6 ms
Wall time: 3min 53s


In [10]:
df.head(10)

,station,year,temp_min
0,147270,1990,-35.0
1,166870,1990,-35.0
2,192830,1952,-35.5
3,166870,1974,-35.6
4,179950,1974,-35.6
5,113410,1954,-36.0
6,179960,1992,-36.1
7,157860,1975,-37.0
8,167860,1972,-37.5
9,169860,2000,-37.6


## Assignment 2

Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month. In this exercise you will use the `temperature-readings.csv` file.

The output should contain the following information:

Year, month, count

In [11]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))
res = res.filter(res['temp'] > 10.0)

res = res.groupBy('year', 'month').count().select('year', 'month', F.col('count')).orderBy(['count', 'year', 'month'], ascending=False)

global df
df = res.toPandas()

CPU times: user 92 ms, sys: 19.8 ms, total: 112 ms
Wall time: 3min 49s


In [12]:
df.head(10)

,year,month,count
0,2014,7,147681
1,2011,7,146656
2,2010,7,143419
3,2012,7,137477
4,2013,7,133657
5,2009,7,133008
6,2011,8,132734
7,2009,8,128349
8,2013,8,128235
9,2003,7,128133


The following configuration is used for both SQL statement.

In [13]:
db_temp = temperature_readings.map(lambda l: l.split(";")) \
            .filter(lambda m: (int(m[1][:4]) >= 1950 and int(m[1][:4]) <= 2014)) \
            .map(lambda m: Row(station = m[0], date = m[1], year = m[1].split("-")[0], month = m[1].split("-")[1],
                               time = m[2], temp = float(m[3]), quality = m[4]))

schema = sc_sql.createDataFrame(db_temp)

schema.registerTempTable("temp_schema")

In [14]:
%%time

global res

res = sc_sql.sql("""
    SELECT year, month, COUNT(year) AS count
    FROM temp_schema
    WHERE year >= 1950 AND year <= 2014 and temp > 10
    GROUP BY year, month
    ORDER BY cast(count as integer) DESC, cast(year as integer) DESC, cast(month as integer) DESC
""")

res = res.toPandas()

CPU times: user 68.4 ms, sys: 25 ms, total: 93.5 ms
Wall time: 4min 24s


In [15]:
res.head(10)

,year,month,count
0,2014,07,147681
1,2011,07,146656
2,2010,07,143419
3,2012,07,137477
4,2013,07,133657
5,2009,07,133008
6,2011,08,132734
7,2009,08,128349
8,2013,08,128235
9,2003,07,128133


Now we repeat the exercise where we take only distinct readings from each station.

In [16]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014) & (res['temp'] > 10.0))

res = res.groupBy('year', 'month', 'station').count().groupBy('year', 'month').count().select('year', 'month', F.col('count')).orderBy(['count', 'year', 'month'], ascending=False)

global df
df = res.toPandas()

CPU times: user 124 ms, sys: 16.7 ms, total: 141 ms
Wall time: 3min 37s


In [17]:
df.head(10)

,year,month,count
0,1972,10,378
1,1973,6,377
2,1973,5,377
3,1973,9,376
4,1972,8,376
5,1972,9,375
6,1972,6,375
7,1972,5,375
8,1971,8,375
9,1972,7,374


In [18]:
%%time

global res

res = sc_sql.sql("""
    SELECT year, month, COUNT(year) AS count
    FROM (
        SELECT year, month, COUNT(year) AS count
        FROM temp_schema
        WHERE year >= 1950 AND year <= 2014 and temp > 10
        GROUP BY year, month, station
    )
    GROUP BY year, month
    ORDER BY cast(count as integer) DESC, cast(year as integer) DESC, cast(month as integer) DESC
""")

res = res.toPandas()

CPU times: user 84.6 ms, sys: 38.7 ms, total: 123 ms
Wall time: 4min 33s


In [19]:
res.head(10)

,year,month,count
0,1972,10,378
1,1973,06,377
2,1973,05,377
3,1973,09,376
4,1972,08,376
5,1972,09,375
6,1972,06,375
7,1972,05,375
8,1971,08,375
9,1972,07,374


## Assignment 3

Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the `temperature-readings.csvfile`.

The output should contain the following information:

Year, month, station number, average monthly temperature

In [20]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1960) & (res['year'] <= 2014))
res = res.groupBy('year', 'month', 'day', 'station')
res = res.agg(F.max('temp').alias('temp_max'), F.min('temp').alias('temp_min'))
res = res.withColumn('davg', (res['temp_max'] + res['temp_min'])/2)
res = res.select('year', 'month', 'day', 'station', 'davg')
res = res.groupBy('year', 'month', 'station').agg(F.mean('davg').alias('average-temperature'))

global df
df = res.orderBy(['average-temperature', 'year', 'month', 'station'], ascending=False).toPandas()

CPU times: user 1.34 s, sys: 121 ms, total: 1.46 s
Wall time: 4min


In [21]:
df.head(10)

,year,month,station,average-temperature
0,2014,7,96000,26.300000
1,1994,7,96550,23.071053
2,1983,8,54550,23.000000
3,1994,7,78140,22.970968
4,1994,7,85280,22.872581
5,1994,7,75120,22.858065
6,1994,7,65450,22.856452
7,1994,7,96000,22.808065
8,1994,7,95160,22.764516
9,1994,7,86200,22.711290


## Assignment 4

Provide  a  list  of  stations  with  their  associated  maximum  measured  temperatures  and maximum  measured  daily  precipitation.  Show  only  those  stations  where  the  maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm. In this exercise you will use the `temperature-readings.csv` and `precipitation-readings.csv` files. 

The output should contain the following information:

Station number, maximum measured temperature, maximum daily precipitation

In [22]:
%%time

res_t = temperature_readings.map(lambda l: l.split(";"))
res_p = precipitation_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda p: Row(year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), station=int(p[0]), precipitation=float(p[3])))
res_p = sc_sql.createDataFrame(res_p)

res_p = res_p.groupBy('year', 'month', 'station', 'day')
res_p = res_p.agg(F.max('precipitation').alias('precip_max'))
res_p = res_p.groupBy('year', 'month', 'station')
res_p = res_p.agg(F.max('precip_max').alias('pmax'), F.min('precip_max').alias('pmin'))
res_p = res_p.filter((res_p['pmin'] >= 100.0) & (res_p['pmax'] <= 200.0))
res_p = res_p.groupBy('station')
res_p = res_p.agg(F.max('pmax').alias('max_daily_precipitation'))

res_t = res_t.map(lambda p: Row(station=int(p[0]), temperature=float(p[3])))
res_t = sc_sql.createDataFrame(res_t)

res_t = res_t.groupBy('station')
res_t = res_t.agg(F.max('temperature').alias('max_temperature'))
res_t = res_t.filter((res_t['max_temperature'] > 25.0) & (res_t['max_temperature'] < 30.0))

global df

df = res_t.join(res_p, "station").orderBy('station', ascending=False).toPandas()

CPU times: user 128 ms, sys: 18.2 ms, total: 146 ms
Wall time: 4min 18s


In [23]:
df.head()

,station,max_temperature,max_daily_precipitation


## Assignment 5

Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate  the  total  monthly  precipitation  for  each  station  before calculating the monthly average (by averaging over stations). In this exercise you will use the `precipitation-readings.csv` and `stations-Ostergotland.csv` files.

HINT (not for the SparkSQL lab): Avoid using joins here! `stations-Ostergotland.csv` is small  and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute `precipitation-readings.csv` then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:

Year, month, average monthly precipitation

In [24]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res_p = res_p.map(lambda p: Row(station=int(p[0]), year=int(p[1].split('-')[0]), month=int(p[1].split('-')[1]), day=int(p[1].split('-')[2]), precipitation = float(p[3])))
res_p = sc_sql.createDataFrame(res_p)
res_p = res_p.filter((res_p['year'] >= 1993) & (res_p['year'] <= 2016))

res = res_p.where(res_p['station'].isin(res_s.value))

res = res.groupBy('year', 'month', 'station').agg(F.sum('precipitation').alias('precipitation_cum'))
res = res.groupBy('year', 'month').agg(F.avg('precipitation_cum').alias('precipitation_avg'))

global df

df = res.orderBy(['year', 'month'], ascending=False).toPandas()

CPU times: user 87.6 ms, sys: 34.2 ms, total: 122 ms
Wall time: 1min 16s


In [25]:
df.head(10)

,year,month,precipitation_avg
0,2016,7,0.0000
1,2016,6,47.6625
2,2016,5,29.2500
3,2016,4,26.9000
4,2016,3,19.9625
5,2016,2,21.5625
6,2016,1,22.3250
7,2015,12,28.9250
8,2015,11,63.8875
9,2015,10,2.2625


## Assignment 6

Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results. 

HINT: The first step is to find the monthly averages for each station. Then, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/DataFrame can beu sed to compute the long-term average by averaging over all the years in the interval.

The out put should contain the following information:

Year, month, difference

In [ ]:
%%time

global res_t
global res_s

res_t = temperature_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res_t = res_t.map(lambda p: Row(year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), day=int(p[1].split("-")[2]), station=int(p[0]), temp=float(p[3])))
res_t = sc_sql.createDataFrame(res_t)
res_t = res_t.filter((res_t['year'] >= 1950) & (res_t['year'] <= 2014))
res_t = res_t.where(res_t['station'].isin(res_s.value))
res_t = res_t.groupBy('year', 'month', 'day', 'station')
res_t = res_t.agg(F.max('temp').alias('temp_max'), F.min('temp').alias('temp_min'))

res_t = res_t.withColumn('davg', (res_t['temp_max'] + res_t['temp_min'])/2)
res_t = res_t.select('year', 'month', 'day', 'station', 'davg')
res_t = res_t.groupBy('year', 'month', 'station').agg(F.mean('davg').alias('davg'))
res_t = res_t.groupBy('year', 'month').agg(F.mean('davg').alias('davg'))

res_t_bigger_range = res_t.filter((res_t['year'] >= 1950) & (res_t['year'] <= 1980))
res_t_bigger_range = res_t_bigger_range.groupBy('month').agg(F.mean('davg').alias('average-temperature'))

res = res_t_bigger_range.join(res_t, "month")
res = res.withColumn('temperature-difference', res['average-temperature'] - res['davg'])
res = res.select('year', 'month', 'temperature-difference')

global df

df = res.orderBy(['year', 'month'], ascending=[0, 0]).toPandas()

In [ ]:
df.head(10)

In [ ]:
# Difference by Year-Month
df['date'] = df['year'].map(str) + "-" + df['month'].map(str)
df.plot.line(x = 'date', y = 'temperature-difference', figsize=(15,10), title = "Difference by Year-Month");

In [ ]:
# Difference by Year
df2 = df.groupby(['year'])['temperature-difference'].mean().to_frame()
df2.reset_index(level=0, inplace=True)
df2.plot.line(x = 'year', y = 'temperature-difference', figsize=(15,10), title = "Difference by Year");

**Comment:**  It does not seem like there is a trend across years which makes sense since we don’t expect any temperature changes in the long-term. However, we can observe that there were some years with especially large (e.g. 1990) and especially small temperature differences (e.g. 1985). Hence, the differences may vary noticeably from year to year.

In [ ]:
# Difference by Month
df3 = df.groupby(['month'])['temperature-difference'].mean().to_frame()
df3.reset_index(level=0, inplace=True)
df3.plot.line(x = 'month', y = 'temperature-difference', figsize=(15,10), title = "Difference by Month");

**Comment:** We can see that the differences in temperature are much larger in the winter months than in the summer months (see “Difference by Month” plot). This makes sense since we computed the difference as: Difference = Östergotland - General. We expect Östergotland to have warmer temperatures in general than more northern parts of Sweden in the Winter period and Östergotland is in the south. Our expectation is fulfilled since the difference is larger in the winter months.

In [ ]:
sc.stop()